In [1]:
import pyspark
from pyspark.sql import SparkSession

import numpy as np
import pandas as pd

In [2]:
spark = SparkSession.builder.master('local[*]').appName('flights').getOrCreate()

# Read data from CSV file
flights = spark.read.csv('./flights.csv', sep=',', header=True, inferSchema=True,
                         nullValue='NA')

# Get number of records
print("The data contain %d records." % flights.count())

# View the first five records
flights.show(5)

# Check column data types
print(flights.printSchema())
print(flights.dtypes)

The data contain 50000 records.
+---+---+---+-------+------+---+----+------+--------+-----+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|
+---+---+---+-------+------+---+----+------+--------+-----+
| 11| 20|  6|     US|    19|JFK|2153|  9.48|     351| NULL|
|  0| 22|  2|     UA|  1107|ORD| 316| 16.33|      82|   30|
|  2| 20|  4|     UA|   226|SFO| 337|  6.17|      82|   -8|
|  9| 13|  1|     AA|   419|ORD|1236| 10.33|     195|   -5|
|  4|  2|  5|     AA|   325|ORD| 258|  8.92|      65| NULL|
+---+---+---+-------+------+---+----+------+--------+-----+
only showing top 5 rows

root
 |-- mon: integer (nullable = true)
 |-- dom: integer (nullable = true)
 |-- dow: integer (nullable = true)
 |-- carrier: string (nullable = true)
 |-- flight: integer (nullable = true)
 |-- org: string (nullable = true)
 |-- mile: integer (nullable = true)
 |-- depart: double (nullable = true)
 |-- duration: integer (nullable = true)
 |-- delay: integer (nullable = true)

None
[('mon', 'int'), 

In [3]:
from pyspark.ml.feature import StringIndexer

flights = StringIndexer(inputCol='org', outputCol='org_idx').fit(flights).transform(flights)

flights.show()

+---+---+---+-------+------+---+----+------+--------+-----+-------+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|org_idx|
+---+---+---+-------+------+---+----+------+--------+-----+-------+
| 11| 20|  6|     US|    19|JFK|2153|  9.48|     351| NULL|    2.0|
|  0| 22|  2|     UA|  1107|ORD| 316| 16.33|      82|   30|    0.0|
|  2| 20|  4|     UA|   226|SFO| 337|  6.17|      82|   -8|    1.0|
|  9| 13|  1|     AA|   419|ORD|1236| 10.33|     195|   -5|    0.0|
|  4|  2|  5|     AA|   325|ORD| 258|  8.92|      65| NULL|    0.0|
|  5|  2|  1|     UA|   704|SFO| 550|  7.98|     102|    2|    1.0|
|  7|  2|  6|     AA|   380|ORD| 733| 10.83|     135|   54|    0.0|
|  1| 16|  6|     UA|  1477|ORD|1440|   8.0|     232|   -7|    0.0|
|  1| 22|  5|     UA|   620|SJC|1829|  7.98|     250|  -13|    4.0|
| 11|  8|  1|     OO|  5590|SFO| 158|  7.77|      60|   88|    1.0|
|  4| 26|  1|     AA|  1144|SFO|1464| 13.25|     210|  -10|    1.0|
|  4| 25|  0|     AA|   321|ORD| 978| 13.75|    

In [4]:
from pyspark.ml.feature import OneHotEncoder

# Create an instance of the one hot encoder
onehot = OneHotEncoder(inputCols=['org_idx'], outputCols=['org_dummy'])

# Apply the one hot encoder to the flights data
onehot = onehot.fit(flights)
flights_onehot = onehot.transform(flights)

# Check the results
flights_onehot.select('org', 'org_idx', 'org_dummy').distinct().sort('org_idx').show()

+---+-------+-------------+
|org|org_idx|    org_dummy|
+---+-------+-------------+
|ORD|    0.0|(7,[0],[1.0])|
|SFO|    1.0|(7,[1],[1.0])|
|JFK|    2.0|(7,[2],[1.0])|
|LGA|    3.0|(7,[3],[1.0])|
|SJC|    4.0|(7,[4],[1.0])|
|SMF|    5.0|(7,[5],[1.0])|
|TUS|    6.0|(7,[6],[1.0])|
|OGG|    7.0|    (7,[],[])|
+---+-------+-------------+



In [5]:
flights_onehot.show(5)

+---+---+---+-------+------+---+----+------+--------+-----+-------+-------------+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|org_idx|    org_dummy|
+---+---+---+-------+------+---+----+------+--------+-----+-------+-------------+
| 11| 20|  6|     US|    19|JFK|2153|  9.48|     351| NULL|    2.0|(7,[2],[1.0])|
|  0| 22|  2|     UA|  1107|ORD| 316| 16.33|      82|   30|    0.0|(7,[0],[1.0])|
|  2| 20|  4|     UA|   226|SFO| 337|  6.17|      82|   -8|    1.0|(7,[1],[1.0])|
|  9| 13|  1|     AA|   419|ORD|1236| 10.33|     195|   -5|    0.0|(7,[0],[1.0])|
|  4|  2|  5|     AA|   325|ORD| 258|  8.92|      65| NULL|    0.0|(7,[0],[1.0])|
+---+---+---+-------+------+---+----+------+--------+-----+-------+-------------+
only showing top 5 rows



In [6]:
from pyspark.sql.functions import round

# Convert 'mile' to 'km' and drop 'mile' column
flights_onehot = flights_onehot.withColumn('km', round(flights_onehot.mile * 1.60934, 0)).drop('mile')

flights_onehot.show(5)

+---+---+---+-------+------+---+------+--------+-----+-------+-------------+------+
|mon|dom|dow|carrier|flight|org|depart|duration|delay|org_idx|    org_dummy|    km|
+---+---+---+-------+------+---+------+--------+-----+-------+-------------+------+
| 11| 20|  6|     US|    19|JFK|  9.48|     351| NULL|    2.0|(7,[2],[1.0])|3465.0|
|  0| 22|  2|     UA|  1107|ORD| 16.33|      82|   30|    0.0|(7,[0],[1.0])| 509.0|
|  2| 20|  4|     UA|   226|SFO|  6.17|      82|   -8|    1.0|(7,[1],[1.0])| 542.0|
|  9| 13|  1|     AA|   419|ORD| 10.33|     195|   -5|    0.0|(7,[0],[1.0])|1989.0|
|  4|  2|  5|     AA|   325|ORD|  8.92|      65| NULL|    0.0|(7,[0],[1.0])| 415.0|
+---+---+---+-------+------+---+------+--------+-----+-------+-------------+------+
only showing top 5 rows



In [7]:
from pyspark.ml.feature import VectorAssembler

# Create an assembler object
assembler = VectorAssembler(inputCols=['km'], outputCol='features')

# Consolidate predictor columns
flights = assembler.transform(flights_onehot)

flights.show(5)

+---+---+---+-------+------+---+------+--------+-----+-------+-------------+------+--------+
|mon|dom|dow|carrier|flight|org|depart|duration|delay|org_idx|    org_dummy|    km|features|
+---+---+---+-------+------+---+------+--------+-----+-------+-------------+------+--------+
| 11| 20|  6|     US|    19|JFK|  9.48|     351| NULL|    2.0|(7,[2],[1.0])|3465.0|[3465.0]|
|  0| 22|  2|     UA|  1107|ORD| 16.33|      82|   30|    0.0|(7,[0],[1.0])| 509.0| [509.0]|
|  2| 20|  4|     UA|   226|SFO|  6.17|      82|   -8|    1.0|(7,[1],[1.0])| 542.0| [542.0]|
|  9| 13|  1|     AA|   419|ORD| 10.33|     195|   -5|    0.0|(7,[0],[1.0])|1989.0|[1989.0]|
|  4|  2|  5|     AA|   325|ORD|  8.92|      65| NULL|    0.0|(7,[0],[1.0])| 415.0| [415.0]|
+---+---+---+-------+------+---+------+--------+-----+-------+-------------+------+--------+
only showing top 5 rows



In [8]:
flights_train, flights_test = flights.randomSplit([0.8, 0.2])

In [9]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

# Create a regression object and train on training data
regression = LinearRegression(featuresCol='features', labelCol='duration').fit(flights_train)

# Create predictions for the test data and take a look at the predictions
predictions = regression.transform(flights_test)
predictions.select('duration', 'prediction').show(5, False)

# Calculate the RMSE
RegressionEvaluator(labelCol='duration', metricName='rmse').evaluate(predictions)

+--------+------------------+
|duration|prediction        |
+--------+------------------+
|370     |345.5798865779781 |
|165     |133.62629361355596|
|115     |133.62629361355596|
|130     |133.62629361355596|
|135     |133.62629361355596|
+--------+------------------+
only showing top 5 rows



17.195229019380108

In [11]:
# Intercept (average minutes on ground)
inter = regression.intercept
print(inter)

# Coefficients
coefs = regression.coefficients
print(coefs)

# Average minutes per km
minutes_per_km = regression.coefficients[0]
print(minutes_per_km)

# Average speed in km per hour
avg_speed = 60 / minutes_per_km
print(avg_speed)

44.28741585397944
[0.0756411540777558]
0.0756411540777558
793.2189921153586


In [10]:
flights_train, flights_test = flights.randomSplit([0.8, 0.2])

# Create a regression object and train on training data
regression = LinearRegression(featuresCol='features', labelCol='duration').fit(flights_train)

# Create predictions for the test data
predictions = regression.transform(flights_test)

# Calculate the RMSE on test data
RegressionEvaluator(labelCol='duration', metricName='rmse').evaluate(predictions)

17.237617586243882

In [13]:
# Create an assembler object
assembler = VectorAssembler(inputCols=['km', 'org_dummy'], outputCol='features')

# Consolidate predictor columns
flights = assembler.transform(flights_onehot)

flights.show(5)

+---+---+---+-------+------+---+------+--------+-----+-------+-------------+------+--------------------+
|mon|dom|dow|carrier|flight|org|depart|duration|delay|org_idx|    org_dummy|    km|            features|
+---+---+---+-------+------+---+------+--------+-----+-------+-------------+------+--------------------+
| 11| 20|  6|     US|    19|JFK|  9.48|     351| NULL|    2.0|(7,[2],[1.0])|3465.0|(8,[0,3],[3465.0,...|
|  0| 22|  2|     UA|  1107|ORD| 16.33|      82|   30|    0.0|(7,[0],[1.0])| 509.0|(8,[0,1],[509.0,1...|
|  2| 20|  4|     UA|   226|SFO|  6.17|      82|   -8|    1.0|(7,[1],[1.0])| 542.0|(8,[0,2],[542.0,1...|
|  9| 13|  1|     AA|   419|ORD| 10.33|     195|   -5|    0.0|(7,[0],[1.0])|1989.0|(8,[0,1],[1989.0,...|
|  4|  2|  5|     AA|   325|ORD|  8.92|      65| NULL|    0.0|(7,[0],[1.0])| 415.0|(8,[0,1],[415.0,1...|
+---+---+---+-------+------+---+------+--------+-----+-------+-------------+------+--------------------+
only showing top 5 rows



In [14]:
flights_train, flights_test = flights.randomSplit([0.8, 0.2])

# Create a regression object and train on training data
regression = LinearRegression(featuresCol='features', labelCol='duration').fit(flights_train)

# Create predictions for the test data
predictions = regression.transform(flights_test)

# Calculate the RMSE on test data
RegressionEvaluator(labelCol='duration', metricName='rmse').evaluate(predictions)

11.390195891247684

In [15]:
# Average speed in km per hour
avg_speed_hour = 60 / regression.coefficients[0]
print(avg_speed_hour)

# Averate minutes on ground at OGG
inter = regression.intercept
print(inter)

# Average minutes on ground at JFK
avg_ground_jfk= inter + regression.coefficients[3]
print(avg_ground_jfk)

# Average minutes on ground at LGA
avg_ground_lga = inter + regression.coefficients[4]
print(avg_ground_lga)

807.7288434941331
15.760271937121509
68.3920951325832
62.79424892951695


In [16]:
from pyspark.ml.feature import Bucketizer

# Create buckets at 3 hour intervals through the day
buckets = Bucketizer(splits=[
    3 * x for x in range(9)
], inputCol='depart', outputCol='depart_bucket')

# Bucket the departure times
bucketed = buckets.transform(flights)
bucketed.select('depart', 'depart_bucket').show(5)

# Create a one-hot encoder
onehot = OneHotEncoder(inputCols=['depart_bucket'], outputCols=['depart_dummy'])

# One-hot encode the bucketed departure times
flights_onehot = onehot.fit(bucketed).transform(bucketed)
flights_onehot.select('depart', 'depart_bucket', 'depart_dummy').show(5)

+------+-------------+
|depart|depart_bucket|
+------+-------------+
|  9.48|          3.0|
| 16.33|          5.0|
|  6.17|          2.0|
| 10.33|          3.0|
|  8.92|          2.0|
+------+-------------+
only showing top 5 rows

+------+-------------+-------------+
|depart|depart_bucket| depart_dummy|
+------+-------------+-------------+
|  9.48|          3.0|(7,[3],[1.0])|
| 16.33|          5.0|(7,[5],[1.0])|
|  6.17|          2.0|(7,[2],[1.0])|
| 10.33|          3.0|(7,[3],[1.0])|
|  8.92|          2.0|(7,[2],[1.0])|
+------+-------------+-------------+
only showing top 5 rows



In [17]:
assembler = VectorAssembler(inputCols=['km', 'org_dummy', 'depart_dummy'], outputCol='features')

flights = assembler.transform(flights_onehot.drop('features'))

flights.show(5)

+---+---+---+-------+------+---+------+--------+-----+-------+-------------+------+-------------+-------------+--------------------+
|mon|dom|dow|carrier|flight|org|depart|duration|delay|org_idx|    org_dummy|    km|depart_bucket| depart_dummy|            features|
+---+---+---+-------+------+---+------+--------+-----+-------+-------------+------+-------------+-------------+--------------------+
| 11| 20|  6|     US|    19|JFK|  9.48|     351| NULL|    2.0|(7,[2],[1.0])|3465.0|          3.0|(7,[3],[1.0])|(15,[0,3,11],[346...|
|  0| 22|  2|     UA|  1107|ORD| 16.33|      82|   30|    0.0|(7,[0],[1.0])| 509.0|          5.0|(7,[5],[1.0])|(15,[0,1,13],[509...|
|  2| 20|  4|     UA|   226|SFO|  6.17|      82|   -8|    1.0|(7,[1],[1.0])| 542.0|          2.0|(7,[2],[1.0])|(15,[0,2,10],[542...|
|  9| 13|  1|     AA|   419|ORD| 10.33|     195|   -5|    0.0|(7,[0],[1.0])|1989.0|          3.0|(7,[3],[1.0])|(15,[0,1,11],[198...|
|  4|  2|  5|     AA|   325|ORD|  8.92|      65| NULL|    0.0|(7,[0],

In [18]:
flights_train, flights_test = flights.randomSplit([0.8, 0.2])

# Train with training data
regression = LinearRegression(labelCol='duration').fit(flights_train)
predictions = regression.transform(flights_test)

RegressionEvaluator(labelCol='duration', metricName='rmse').evaluate(predictions)

# Average minutes on ground at OGG for flights departing between 21:00 and 24:00
avg_eve_ogg = regression.intercept
print(avg_eve_ogg)

# Average minutes on ground at OGG for flights departing between 00:00 and 03:00
avg_night_ogg = regression.intercept + regression.coefficients[8]
print(avg_night_ogg)

# Average minutes on ground at JFK for flights departing between 00:00 and 03:00
avg_night_jfk = regression.intercept + regression.coefficients[3] + regression.coefficients[8]
print(avg_night_jfk)

10.44349647064473
-4.583695729211874
47.25590900637394


In [19]:
# Find the RMSE on testing data
from pyspark.ml.evaluation import RegressionEvaluator
rmse = RegressionEvaluator(labelCol='duration').evaluate(predictions)
print("The test RMSE is", rmse)

# Average minutes on ground at OGG for flights departing between 21:00 and 24:00
avg_eve_ogg = regression.intercept
print(avg_eve_ogg)

# Average minutes on ground at OGG for flights departing between 03:00 and 06:00
avg_night_ogg = regression.intercept + regression.coefficients[9]
print(avg_night_ogg)

# Average minutes on ground at JFK for flights departing between 03:00 and 06:00
avg_night_jfk = regression.intercept + regression.coefficients[9] + regression.coefficients[3]
print(avg_night_jfk)

The test RMSE is 11.492471974857887
10.44349647064473
10.972701346575596
62.812306082161406


In [20]:
onehot = OneHotEncoder(inputCols=['dow'], outputCols=['dow_dummy'])
flights = onehot.fit(flights).transform(flights)

onehot = OneHotEncoder(inputCols=['mon'], outputCols=['mon_dummy'])
flights = onehot.fit(flights).transform(flights)

flights.show(5)

+---+---+---+-------+------+---+------+--------+-----+-------+-------------+------+-------------+-------------+--------------------+-------------+--------------+
|mon|dom|dow|carrier|flight|org|depart|duration|delay|org_idx|    org_dummy|    km|depart_bucket| depart_dummy|            features|    dow_dummy|     mon_dummy|
+---+---+---+-------+------+---+------+--------+-----+-------+-------------+------+-------------+-------------+--------------------+-------------+--------------+
| 11| 20|  6|     US|    19|JFK|  9.48|     351| NULL|    2.0|(7,[2],[1.0])|3465.0|          3.0|(7,[3],[1.0])|(15,[0,3,11],[346...|    (6,[],[])|    (11,[],[])|
|  0| 22|  2|     UA|  1107|ORD| 16.33|      82|   30|    0.0|(7,[0],[1.0])| 509.0|          5.0|(7,[5],[1.0])|(15,[0,1,13],[509...|(6,[2],[1.0])|(11,[0],[1.0])|
|  2| 20|  4|     UA|   226|SFO|  6.17|      82|   -8|    1.0|(7,[1],[1.0])| 542.0|          2.0|(7,[2],[1.0])|(15,[0,2,10],[542...|(6,[4],[1.0])|(11,[2],[1.0])|
|  9| 13|  1|     AA|   419|

In [21]:
assembler = VectorAssembler(inputCols=[
    'km', 'org_dummy', 'depart_dummy', 'dow_dummy', 'mon_dummy'
], outputCol='features')

flights = assembler.transform(flights.drop('features'))
flights.show(5)

+---+---+---+-------+------+---+------+--------+-----+-------+-------------+------+-------------+-------------+-------------+--------------+--------------------+
|mon|dom|dow|carrier|flight|org|depart|duration|delay|org_idx|    org_dummy|    km|depart_bucket| depart_dummy|    dow_dummy|     mon_dummy|            features|
+---+---+---+-------+------+---+------+--------+-----+-------+-------------+------+-------------+-------------+-------------+--------------+--------------------+
| 11| 20|  6|     US|    19|JFK|  9.48|     351| NULL|    2.0|(7,[2],[1.0])|3465.0|          3.0|(7,[3],[1.0])|    (6,[],[])|    (11,[],[])|(32,[0,3,11],[346...|
|  0| 22|  2|     UA|  1107|ORD| 16.33|      82|   30|    0.0|(7,[0],[1.0])| 509.0|          5.0|(7,[5],[1.0])|(6,[2],[1.0])|(11,[0],[1.0])|(32,[0,1,13,17,21...|
|  2| 20|  4|     UA|   226|SFO|  6.17|      82|   -8|    1.0|(7,[1],[1.0])| 542.0|          2.0|(7,[2],[1.0])|(6,[4],[1.0])|(11,[2],[1.0])|(32,[0,2,10,19,23...|
|  9| 13|  1|     AA|   419|

In [22]:
flights_train, flights_test = flights.randomSplit([0.8, 0.2])

# Fit linear regressino model to training data
regression = LinearRegression(labelCol='duration').fit(flights_train)

# Make predictions on test data
predictions = regression.transform(flights_test)

# Calculate the RMSE on test data
rmse = RegressionEvaluator(labelCol='duration', metricName='rmse').evaluate(predictions)
print("The test RMSE is", rmse)

# Look at the model coefficients
coeffs = regression.coefficients
print(coeffs)

The test RMSE is 10.931751188224863
[0.07443512743688485,27.36855888894463,20.3316642325607,52.00785683481466,45.973008510822986,17.734585003216186,15.190870783774841,17.400917630454682,-15.333287586578704,0.26260939301640696,3.8773967064490042,6.6549610399540615,4.530231748525794,8.67046361437582,8.599210337273902,0.35503815432749486,0.03780934659698205,-0.31697251818703565,0.1269933758846764,0.17562101269224617,0.00780243002593114,-2.188477821273564,-2.5235383057641805,-2.2154499015131637,-3.761226214256012,-4.404697293365317,-4.444767382437013,-4.722359644740087,-4.633022590801523,-4.088521423959229,-2.9871102606417033,-1.123108135308757]


In [23]:
# Fit Lasso model (α = 1) to training data
regression = LinearRegression(labelCol='duration', regParam=1, elasticNetParam=1).fit(flights_train)
predictions = regression.transform(flights_test)

# Calculate the RMSE on testing data
rmse = RegressionEvaluator(labelCol='duration', metricName='rmse').evaluate(predictions)
print("The test RMSE is", rmse)

# Look at the model coefficients
coeffs = regression.coefficients
print(coeffs)

# Number of zero coefficients
zero_coeff = sum([beta == 0 for beta in regression.coefficients])
print("Number of coefficients equal to 0:", zero_coeff)

The test RMSE is 11.87351279852486
[0.07353801111435675,5.403895002115591,0.0,29.020811197072142,21.886161591916714,0.0,-2.34922721349363,0.0,0.0,0.0,0.0,0.0,0.0,1.0978146110257674,1.1882834342802642,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
Number of coefficients equal to 0: 25
